In [12]:
import torch
import torch.nn as nn
from torchvision import transforms
from network import resnet
from src import dataset
from tqdm import tqdm
from torch.optim import adam

In [13]:
num_classes = 1
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
network = resnet.resnet18(num_classes).to(device)
loss = nn.BCELoss().to(device)
transform =transforms.Compose([transforms.ToTensor()])
optimizer = adam.Adam(network.parameters(), lr=0.001, weight_decay=0.0001)

In [14]:
data_dir = "src/dataset/"
myDataSet = dataset.MyDataSet(data_dir, transform=transform)
total_ratio = 0.2
train_ratio = 0.1
val_ratio = 0.05
test_ratio = total_ratio - train_ratio - val_ratio

dataset_size = len(myDataSet)
train_size = int(dataset_size * train_ratio)
val_size = int(dataset_size * val_ratio)
test_size = int(dataset_size * test_ratio)
residual_size = dataset_size - train_size - val_size - test_size
train_dataset, val_dataset, test_dataset, _ = torch.utils.data.random_split(myDataSet, [train_size, val_size, test_size, residual_size])


In [15]:
batch_size = 16
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [16]:
epoch = 5
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
network.apply(init_weights)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
for _ in range(1):
    train_loss = 0.0
    for image, label in tqdm(train_dataloader):
        image = image.to(device)
        label = label.to(device).float()
        
        output = network(image)
        output = torch.sigmoid(output)
        
        batch_loss = loss(output, label)
        
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        train_loss += batch_loss
    cost = train_loss/len(train_dataloader)
    print(cost)

100%|██████████| 441/441 [17:34<00:00,  2.39s/it]  

tensor(0.3525, grad_fn=<DivBackward0>)


In [21]:
with torch.no_grad():
    test_loss = 0.0
    for image, label in tqdm(test_dataloader):
        image = image.to(device)
        label = label.to(device).float()
        
        test_output = network(image)
        test_output = torch.sigmoid(test_output)
        
        batch_test_loss = loss(test_output, label)
        
        test_loss += batch_test_loss
    print(test_loss/len(test_dataloader))

100%|██████████| 220/220 [03:16<00:00,  1.12it/s]

tensor(0.3371)
